In [2]:
import pandas as pd
import xml.etree.ElementTree as ET
import mysql.connector

In [ ]:
# Conectar ao banco de dados MySQL
conexao = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Sen3180326#*#',
    database='labstag'
)

cursor = conexao.cursor()
print(conexao.is_connected())  # Deve retornar True se a conexão estiver ativa

In [ ]:
# Ler o arquivo XML
tree = ET.parse("biblia.xml")
root = tree.getroot()

# Criar lista para armazenar os versículos
dados = []

In [ ]:
# Percorrer o XML e extrair informações
for book in root.findall("book"):
    nome_livro = book.get("name")
    abbrev = book.get("abbrev")
    total_capitulos = book.get("chapters")

    for chapter in book.findall("c"):
        cap_num = chapter.get("n")

        for verse in chapter.findall("v"):
            versiculo_num = verse.get("n")
            texto = verse.text

            dados.append([nome_livro, abbrev, total_capitulos, cap_num, versiculo_num, texto])

In [ ]:
# Criar DataFrame no Pandas
df = pd.DataFrame(dados, columns=["name", "abbrev", "chapters", "chapter_num", "verse_num", "text"])

# Verificar se os dados foram carregados corretamente
print(df.head())  # Mostrar as primeiras 5 linhas do DataFrame
print(f"Total de linhas: {len(df)}")

In [ ]:
# Criar nova tabela no banco de dados (se necessário)
cursor.execute("""
    CREATE TABLE IF NOT EXISTS versiculos (
        id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(255),
        abbrev VARCHAR(15),
        chapters INT,
        chapter_num INT,
        verse_num INT,
        text TEXT
    );
""")

In [ ]:
# Inserir os dados no MySQL
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO versiculos (name, abbrev, chapters, chapter_num, verse_num, text) 
        VALUES (%s, %s, %s, %s, %s, %s)
    """, tuple(row))

In [ ]:
# Criar índices para otimizar buscas
cursor.execute("CREATE INDEX idx_livro ON versiculos(name);")
cursor.execute("CREATE INDEX idx_abbrev ON versiculos(abbrev);")
cursor.execute("CREATE INDEX idx_capitulo ON versiculos(chapter_num);")
cursor.execute("CREATE INDEX idx_versiculo ON versiculos(verse_num);")

In [ ]:
# Commit e fechar conexão
conexao.commit()
cursor.close()
conexao.close()